# 경기 결과를 기록하면 이를 CSV 파일로 변환시킨다.

## 이후 더 좋은 방법이 있다면 자동화를 할 수 있도록 한다.

In [19]:
# 모듈 import

from bs4 import BeautifulSoup
import re

from selenium import webdriver
from datetime import datetime

from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

import folium

In [20]:
def select_inning(inning_index): # 이닝 버튼 클릭하는 함수
    try:
        # 모든 이닝 버튼 가져오기
        inning_buttons = wd.find_elements(By.CLASS_NAME, 'SetTab_button_tab___H3-s')

        # 원하는 이닝 클릭
        inning_buttons[inning_index].click()

        # aria-selected 속성이 true로 바뀔 때까지 대기
        time.sleep(1)
        selected = inning_buttons[inning_index].get_attribute("aria-selected")
        while selected != "true":
            time.sleep(1)
            selected = inning_buttons[inning_index].get_attribute("aria-selected")

        return 1

    except IndexError:
        return 0

    except Exception as e:
        print(f"An error occurred: {e}")
        return -1

In [21]:
# 이닝 데이터 가져오는 함수
def get_inning_data():
    try:
        # 이닝 데이터 찾기 (여기서는 클래스 이름 예시)
        # play_by_play = wd.find_elements(By.CLASS_NAME, 'RelayInfo_relay_info_group__1aqtH')  # 요소의 클래스 이름 필요
        play_by_play = wd.find_elements(By.CSS_SELECTOR, '[class ^= "RelayInfo_relay_info_group"]')
        for play in play_by_play:
            soupInning = BeautifulSoup(play.get_attribute('innerHTML'), 'html.parser')
            return soupInning
    except Exception as e:
        print(f"Error fetching inning data: {e}")

In [22]:
# HTML을 한 줄씩 읽어들여서 태그로 시작하지 않는 줄만 return
def non_tag_lines(html_string):

    result = []
    
    # HTML을 줄 단위로 분리
    lines = html_string.splitlines()
    
    # 태그로 시작하는지 여부를 판단하는 정규 표현식
    tag_pattern = re.compile(r'^\s*<[^>]+>')

    for line in lines:
        # 태그로 시작하지 않는 줄만 출력
        if not tag_pattern.match(line):
            result.append(line.strip())  # strip()으로 양 끝의 공백 제거
    
    return result

In [27]:
def make_list(text) :

    raw_data = []
    turn_data = []
    start_found = False

    # 이닝 소개를 찾기 위한 패턴
    pattern_inning = re.compile(r'\d+회\w - \w+공격')

    # 한 턴을 찾기 위한 패턴 (턴 : 한 타석)
    start_marker = "투구 위치 보기"
    end_marker_pattern = re.compile(r'\w+ 선수 페이지')
    end_marker2 = pattern_inning

    for line in text :
        if pattern_inning.match(line) :
            raw_data.append(line)
        elif start_marker in line :
            start_found = True
            continue
        elif end_marker_pattern.match(line) or pattern_inning.match(line):
            if start_found :
                item = ' '.join(turn_data)
                raw_data.append(item)
                turn_data = []
                start_found = False
        elif start_found:
            turn_data.append(line.strip())
    
    return raw_data



In [24]:
# Event.json 생성

import json
from collections import OrderedDict

def get_event(soup) :
    formatted_soup = soup.prettify()
    lines = non_tag_lines(formatted_soup) # 경기 내용만 추출
    raw = make_list(lines) # 초기 데이터 생성
    print(raw)

    # 태그를 제외한 문자열만 골라내기
    # event_data = OrderedDict()
    # inning_first = formatted_soup.select_one('[class^="OffenseTitle_title__"]')
    # inning_second = soup.select_one('[class^="OffenseTitle_title__"]')
    # print(inning_first)
    # print(inning_second
    # strong_tag = formatted_soup.find('strong', class_='OffenseTitle_title__')

    # # 텍스트 추출
    # if strong_tag:
    #     text = strong_tag.get_text(strip=True)  # strip=True로 앞뒤 공백 제거
    #     print(text))

In [25]:
# URL 구성하기

base_URL = "https://m.sports.naver.com/game/"
date = "20240908"
# date = datetime.today().strftime("%Y%m%d")

team = input("팀 이름")
foot_URL = "02024/relay"

URL = base_URL + date + team + foot_URL

In [28]:
# URL 접속 후 데이터 추출하기
wd = webdriver.Chrome(service = Service())
            
wd.get(URL)
time.sleep(1)  #웹페이지 연결할 동안 1초 대기

# 이닝 순서에 따라 클릭하고 데이터 수집
for inning_index in range(0, 13): # 최대 12회까지 진행
    flag = 0
    flag = select_inning(inning_index) # 이닝을 바꾸는 함수
    if flag == 1 :
        soupInning = get_inning_data() # 이닝 데이터 가져오는 함수
        get_event(soupInning)
        
# 브라우저 종료
wd.quit()


['1회말 - LG공격', '박동원 : 우익수 플라이 아웃 기록 펼치기 LG 승리 확률 50.0% (-6.7%p) 3 구 타격 122Km/h 슬라이더 2 구 헛스윙 124Km/h 슬라이더 볼카운트 0 - 2 1 구 스트라이크 112Km/h 커브 볼카운트 0 - 1 오지환 선수 페이지', '오지환 : 볼넷 기록 펼치기 LG 승리 확률 56.7% (+1.5%p) 7 구 볼 121Km/h 슬라이더 6 구 파울 119Km/h 체인지업 볼카운트 3 - 2 1루주자 김현수 : 도루로 2루까지 진루 5 구 볼 115Km/h 커브 볼카운트 3 - 2 4 구 파울 142Km/h 직구 볼카운트 2 - 2 3 구 스트라이크 120Km/h 체인지업 볼카운트 2 - 1 2 구 볼 121Km/h 슬라이더 볼카운트 2 - 0 1 구 볼 123Km/h 슬라이더 볼카운트 1 - 0 문보경 선수 페이지', '문보경 : 삼진 아웃 기록 펼치기 LG 승리 확률 54.5% (-5.6%p) 4 구 헛스윙 142Km/h 직구 3 구 파울 118Km/h 체인지업 볼카운트 1 - 2 2 구 파울 123Km/h 슬라이더 볼카운트 1 - 1 1 구 볼 116Km/h 커브 볼카운트 1 - 0 오스틴 선수 페이지', '오스틴 : 삼진 아웃 기록 펼치기 LG 승리 확률 60.1% (-4.8%p) 4 구 스트라이크 142Km/h 직구 3 구 스트라이크 115Km/h 커브 볼카운트 1 - 2 2 구 스트라이크 120Km/h 체인지업 볼카운트 1 - 1 1 구 볼 120Km/h 체인지업 볼카운트 1 - 0 김현수 선수 페이지', '2루주자 홍창기 : 3루까지 진루 김현수 : 우익수 앞 1루타 기록 펼치기 LG 승리 확률 64.9% (+4.8%p) 3 구 타격 140Km/h 직구 2 구 스트라이크 120Km/h 체인지업 볼카운트 1 - 1 1 구 볼 113Km/h 커브 볼카운트 1 - 0 홍창기 선수 페이지', '1회초 - 한화공격', '홍창기 : 좌중간 2루타 기록 펼치기 LG 승리 확률 60.1% (+5.0%p) 4